# Домашняя работа часть 2  (5 баллов): совмещение спарсификации и квантизации.


В этом задании вам предстоит совместить спарсификацию и квантизацию.

В это заданаии мы предлагаем вам самим реализовать функции квантизации и спарсисикации и далее совместить их вместе. В качестве базовых методов предлагаем взять GPTQ и SparseGPT, использование других методов не исключается.

Функция квантизации также должна возвращать уровень сжатия модели, считаем что понижение битовости в два раза соотвествует сжатию в два раза,  зануление половины весов так же соответствует сжатию в два раза.

Что бы упросить задание, мы не будем квантизовать всю модель, а квантизуем только по слоям.

Резальтат работы это метрики L1 и L2  для $|C(W)X^T - WX^T|$ для каждого слоя, $С$  - функция одновременной спарсификации и квантизации.


В папке llama7b_weights содержатся веса для одоного слоя LLaMа7b и сопутствующие активации в папке llama7b_act_scales (аггрегировые по датасету)


В качестве результата нужно будет сдать ноутбук с решением и метриками.

### Баллы

Релизация алгортима спарсификации - 1 балл
Релизация алгортима квантизации - 1 балл
Объеденение двух алгоритмов - 3 балла

In [20]:
import os
import torch 

In [112]:
# load weights

FOLDER = "../llama7b_weights/"
names = os.listdir(FOLDER)
weight_paths =  {name.replace(".pt",'') :os.path.join(FOLDER, name) for name in names}
names = [name.replace(".pt",'') for name in names]
W = torch.load(weight_paths[names[1]])
W.shape

torch.Size([4096, 11008])

In [113]:
# Расчет матрицы гессе для слоя
X = torch.load("../llama7b_act_scales/Llama-2-7b-hf.pt")[names[1]]

H = torch.outer(X, X)

damp = 0.01 * torch.mean(torch.diag(H))
diag = torch.arange(X.shape[0])
H[diag, diag] += damp

H = torch.linalg.cholesky(H)
H = torch.cholesky_inverse(H)
H = torch.linalg.cholesky(H, upper=True)
Hinv = H
H.shape

torch.Size([11008, 11008])

Для вдохновения рекомендуем воспользоваться кодов QUIK

https://github.com/IST-DASLab/QUIK/blob/9558d7121c698174fc93940e0b52c38c746c97ea/experiments/quik_utils.py#L80

# Ваше решение тут ...

#### Пример расчета ошибки

In [124]:
#  Фкнкцию ниже можно переписать для своего удобства 

def test(your_quantization_function):
    c = your_quantization_function
    result = dict()
    for name in names:
        X = torch.load("../llama7b_act_scales/Llama-2-7b-hf.pt")[name]
        W = torch.load(weight_paths[name])
        X = X.float()
        W = W.float()
        WQ, compress_size = c(W)
        l2 = ((W@X.T - WQ@X.T)**2).mean()
        l1 = (W@X.T - WQ@X.T).abs().mean()
        result[name] = (l2,l1, compress_size)
    return  result


def dummy_compress(X):
    X = torch.round(X)
    compress_size = 0
    return X, compress_size


test(dummy_compress)

{'model.layers.20.self_attn.k_proj': (tensor(11.6558), tensor(2.6132), 0),
 'model.layers.20.mlp.down_proj': (tensor(158.4233), tensor(10.0173), 0),
 'model.layers.20.mlp.gate_proj': (tensor(4.2363), tensor(1.6319), 0),
 'model.layers.20.self_attn.o_proj': (tensor(4.2832), tensor(1.6623), 0),
 'model.layers.20.self_attn.v_proj': (tensor(4.8678), tensor(1.7560), 0),
 'model.layers.20.mlp.up_proj': (tensor(3.5155), tensor(1.4972), 0),
 'model.layers.20.self_attn.q_proj': (tensor(10.3696), tensor(2.4367), 0)}